In [1]:
import pickle
import copy
import torch
import time
import numpy as np
np.set_printoptions(precision = 4)



from datasets.dataset import _load_collection_real,_load_collection_syn,_load_collection_bbq_data
from models_utility.construct_models import _initialize_SMkernel_hyp,_initialize_SMkernel_hyp_v2,_make_gpmodel
from utility.eval_metric import _evaluate_metric,_write_summary
import matplotlib.pyplot as plt
import random
from kernels.RBF_kernel import RBF
from kernels.SM_kernel import SM



from datasets.dataset import _load_collection_real,_load_collection_syn,_load_collection_bbq_data,_load_collection_ssdkl_data
from models_utility.construct_models import _initialize_SMkernel_hyp,_initialize_SMkernel_hyp_v2,_make_gpmodel
from utility.eval_metric import _evaluate_metric,_write_summary
import matplotlib.pyplot as plt





In [ ]:
import seaborn as sns
sns.set()
sns.set_style('white')
#sns.set_style('whitegrid')
import matplotlib
plt.rcParams["axes.grid"] = True
plt.rc('font', family='serif')
#plt.rc('font', weight='bold')

current_palette = sns.color_palette(sns.hls_palette(10+1, l=.4, s=1.0))
sns.palplot(current_palette)
current_palette = np.asarray(current_palette)

In [ ]:
import os.path
from os import path
save_figure_path  = './jupyters/result_figure/exp1-1/'
os.makedirs(save_figure_path) if path.isdir(save_figure_path) is False else 1

In [ ]:
random_seed = 1000
device = True



#['co2','airline'],['Concrete','Noise'],['rubber','pores','tread']
#filename = 'co2'


#filename = 'Concrete' # more possible
#filename = 'Noise'

#filename = 'rubber'
#filename = 'tread'
#filename = 'pores' #does not work

#x_train, x_test, y_train, y_test = _load_collection_bbq_data(filename , cuda_option=device)



#filename = 'skillcraft'
filename = 'parkinsons'
#filename = 'elevators'
#filename = 'protein'
#filename = 'blog'
#filename = 'ctslice'
#filename = 'buzz'  #50000
#filename = 'electric'  #50000
x_train, x_test, y_train, y_test = _load_collection_ssdkl_data(filename,random_seed=random_seed, numtotal = 200000, cuda_option = device)




setting_dict = {}
setting_dict['random_seed'] = random_seed
setting_dict['input_dim'] = x_train.shape[1]

setting_dict['init_sample_num'] = 100
setting_dict['Num_Q'] = 5
setting_dict['lr_hyp'] = 1e-2         #0.001,0.0005  error -- >0.0001 equal_reg
setting_dict['noise_err'] = .1*y_train.cpu().data.numpy().std()
#setting_dict['noise_err'] = .1

#setting_dict['sampling_option'] = 'weight'
#setting_dict['sampling_option'] = 'uniform'
setting_dict['kl_option'] = False  #['True','False']
setting_dict['num_sample_pt'] = 5
setting_dict['num_batch'] = 1
setting_dict['tau0'] = 1000
setting_dict['iter'] = 2000        #[5000,8000]
setting_dict['num_rep'] = 2
setting_dict['weight_rate'] = 1e-2  # [1e-2,1e-3,1e-5]

setting_dict['optimizer'] = 'Adam'   #['SGD','Adam']




In [ ]:
setting_dict = _initialize_SMkernel_hyp_v2(x_train, y_train, setting_dict, random_seed )

gpr_vfe = _make_gpmodel(model_name='gpvfe', setting_dict =setting_dict, device=device)
gpr_vfe._set_data(batch_x=x_train, batch_y=y_train)
gpr_vfe._set_inducing_pt(2*setting_dict['Num_Q']*setting_dict['num_sample_pt'] )

In [ ]:
   for i in range(setting_dict['iter'] + 1):

        gpr_vfe.train()
        ith_tic = time.time()
        ith_loss = gpr_vfe.compute_loss(batch_x=x_train, batch_y=y_train, kl_option=setting_dict['kl_option'])
        ith_loss.backward()



#         ################################################################
#         if ith_model_name not in ['equal_reg_nat','weight_reg_nat'] and setting_dict['optimizer'] is not 'Adam':
#             # torch_utils.clip_grad_norm_([ith_model.mu,ith_model.std ],
#             #                             max_grad_norm)

#             torch_utils.clip_grad_norm_([ith_model.mu],
#                                         max_grad_norm,norm_type='inf')

#             torch_utils.clip_grad_norm_([ith_model.std],
#                                         max_grad_norm, norm_type='inf')


#         ################################################################
        gpr_vfe.optimizer.step()
        gpr_vfe.optimizer.zero_grad()
        torch.cuda.empty_cache()

        ith_toc = time.time()

        with torch.no_grad():
            #pred_train_mu, pred_train_var = ith_model._predict(inputs_new=x_train)
            pred_test_mu, pred_test_var = gpr_vfe._predict(inputs_new=x_test)
            ith_rmse,ith_mnll = _evaluate_metric(pred_test_mu, pred_test_var, y_test)
            
        if i%100 ==0:
            print('%d th loss: %.3f, mse : %.3f,  mnll : %.3f' % (i,ith_loss.cpu().data.numpy(), ith_rmse, ith_mnll))

